In [32]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [33]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [34]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [35]:
tables = soup.find_all('table',class_='wikitable sortable')
len(tables)
type(tables[0])

bs4.element.Tag

In [36]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [37]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [38]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241]
270,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242]
271,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243]
272,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244]


In [39]:
df_2020 = df[['Title','Cast and crew']]

In [40]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [41]:
!pip install tmdbv3api

In [42]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '9230b416f1da187d1d579cb81aa35889'

In [43]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [44]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

C:\Users\upend\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [46]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [47]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\upend\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [49]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\upend\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [51]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\upend\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [53]:

df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [54]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [55]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [56]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [57]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
269,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes
270,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure,News of the World
271,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
272,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [58]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [59]:
new_df20 = new_df20.dropna(how='any')

In [60]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

C:\Users\upend\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
268,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Comedy Fantasy Family,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
269,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
271,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
272,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [62]:
old_df = pd.read_csv('final_data.csv')

In [63]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,toy story,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,grumpier old men,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,waiting to exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,father of the bride part ii,Steve Martin Diane Keaton Martin Short Charles...
...,...,...,...,...,...,...,...
36799,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
36800,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
36801,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller,1917,George MacKay Dean-Charles Chapman Mark Strong...
36802,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [64]:
final_df = old_df.append(new_df20,ignore_index=True)

In [65]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,toy story,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,grumpier old men,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,waiting to exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,father of the bride part ii,Steve Martin Diane Keaton Martin Short Charles...
...,...,...,...,...,...,...,...
37045,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Comedy Fantasy Family,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
37046,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
37047,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
37048,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [66]:
final_df.to_csv('final_data2.csv',index=False)